# Comparison on difficult cases

With this notebook, you can compare how changes to OpenSTEF improve it's performance on concrete cases where the performance of OpenSTEF was not that great. 

At Alliander, the operations team identified a number of cases where improvements of the forecast accuracy would be very valuable.
These cases are stored as 'fixed' train/test data experiments that can be used to systematically measure improvement ideas to OpenSTEF.

This notebook uses the `train_pipeline_common` function from OpenSTEF, which allows detailed modifications of the input data.

The notebook is structured as follows:
- Generate benchmark (needed once)
  - For each experiment:
    - read data
    - train model
    - generate forecast
  - Calculate performance metrics
  - (Optionally: repeat N times to make outcome more robust)
- Test improvement idea (import local OpenSTEF)
  - For each experiment:
    - read data
    - train model
    - generate forecast
  - Calculate performance metrics
  - (Optionally: repeat N times to make outcome more robust)
- Compare benchmark and improvement idea

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
from tqdm.notebook import tqdm
import plotly.express as px
from typing import Dict, List
from sklearn import metrics
import plotly.subplots as subplots
import scipy.signal as signal
from sklearn import metrics
import plotly.express as px
import plotly.graph_objects as go

# Set plotly as the default pandas plotting backend
pd.options.plotting.backend = "plotly"
import plotly.io as pio

pio.renderers.default = "plotly_mimetype+notebook"

# Import required stuff from OpenSTEF
from openstef.data_classes.prediction_job import PredictionJobDataClass
from openstef.data_classes.model_specifications import ModelSpecificationDataClass

from openstef.metrics.figure import plot_feature_importance
from openstef.pipeline.train_model import train_model_pipeline
from openstef.pipeline.create_forecast import create_forecast_pipeline

## Settings

In [ ]:
difficult_location_number = 1  # 1, 2 of 3
quantiles = [0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95]

PLOT_POSITIVE_PEAKS = True
PLOT_NEGATIVE_PEAKS = False
compare_positive_peaks = True  # For quantile peak detection evaluation

PLOT_ROLLING_WINDOW = False

percentile = 50  # Select forecast percentile to evaluate

# Less important settings:

# For rolling window calculations. 1 sample = window of 15 minutes
window_size = 4 * 24

# Peak detection settings
peak_evaluation_width = 10
peak_miss_threshold = 1e6
load_threshold_min_quantile = 0.1
load_threshold_max_quantile = 0.9
peak_distance = 4 * 3  # 3 hours

fft_window_size = int(7 * 24 * 60 / 15) # 7 days
fft_forecast_N = 192 # 2 days
fft_horizons = [0, 2, 4, 48, 96, 192]

In [ ]:
from typing import List
import scipy.fft as sf
import numpy as np

def fourier_forecast(
        signal: np.ndarray,
        top_K: int = 5,
        forecast_N: int = 192,
):
    N = signal.shape[0]
    X = sf.rfft(signal) / N
    X_thresholded = np.copy(X)
    top_indices = np.argsort(abs(X_thresholded))[::-1][top_K:]
    X_thresholded[top_indices] = 0
    
    # Generate restored signal
    frequencies = np.fft.rfftfreq(N, d=1.0)
    t = np.arange(0, N + forecast_N)
    signal_restored = np.zeros(t.size)
    for i in range(len(X_thresholded)):
        amplitude = np.absolute(X_thresholded[i]) / N
        phase = np.angle(X_thresholded[i])
        signal_restored += amplitude * np.cos(2 * np.pi * frequencies[i] * t + phase)
    
    # Normalize the signal
    signal_restored = (signal_restored / abs(signal_restored).max()) * signal.max()
    
    return signal_restored

# The following function adds a fourier forecast feature to the dataset
# It takes an arbitrary long signal, a window size and forecast horizons
# For each point in the signal, it calculates the fourier forecast using the previous window

def fourier_forecast_feature(
        signal: np.ndarray,
        window_size: int,
        horizons: List[int],
        top_K: int = 5,
):
    N = signal.shape[0]
    forecast_features = np.zeros((N, len(horizons)))
    forecast_features[:, :] = np.nan
    
    for i in range(N):
        if i < window_size:
            continue
            
#         signal_window = signal[i - window_size+1:i+1] # +1 to include the current point
        signal_window = signal[i - window_size:i] # +1 to include the current point
        forecast = fourier_forecast(signal_window, top_K=top_K, forecast_N=max(horizons) + 1)[window_size:]
        
        for j, horizon in enumerate(horizons):
            forecast_features[i, j] = forecast[horizon]
            
    return forecast_features


## Generate Benchmark
Generate a benchmark using the latest OpenSTEF. Only needed once.

In [ ]:
# Load experiment
def get_experiments(difficult_location_number: int = 1, n_days: int = 10):
    """
    Returns a list of dicts, with each dict being an experiment with:
    - name
    - dataset
    - test_length # length counted from the end of the data that will be considered the Test period
    -"""
    complete_data = pd.read_csv(
        f"data/model_input_difficult_location_{difficult_location_number}.csv",
        parse_dates=True,
        index_col=0,
    )[:-3]

    def create_dataset(i: int):
        dataset = complete_data.iloc[0 * 24 * 4: (120 + i) * 24 * 4 + 24 * 4]
        dataset = dataset[~dataset["load"].isna()]
        
        fft_features = fourier_forecast_feature(
            dataset["load"].values,
            window_size=fft_window_size,
            horizons=fft_horizons,
            top_K=10,
        )
        for horizon in fft_horizons:
            dataset[f"fft_{horizon}"] = fft_features[:, fft_horizons.index(horizon)]
        
        dataset = dataset.iloc[fft_window_size:]
        
        return dataset

    # Generate the experiments where we forecast for 1 day each time
    # Dataset contains both the 120 days of training data + the 1 day of test data
    result = [
        dict(  # Predict day 1
            name=f"N{i + 1}_loc{difficult_location_number}",
            dataset=create_dataset(i),
            test_length=24 * 4 if i != n_days - 1 else 24 * 4 - 3,
        )
        for i in range(10)
    ]
    return result


In [ ]:
experiments = get_experiments(difficult_location_number)

In [ ]:
# Shared prediction job specs
pj = dict(
    id=1,  # Should be updated for each experiment
    model="xgb",
    quantiles=quantiles,
    name="benchmark",
)

# These are not relevant...
pj_specs_that_should_be_optional_in_future_openstef_versions = dict(
    forecast_type="demand",
    lat=52.0,
    lon=5.0,
    horizon_minutes=47 * 60,
    description="description",
    resolution_minutes=15,
    hyper_params={
        
    },
    feature_names=None,
)

pj.update(pj_specs_that_should_be_optional_in_future_openstef_versions)

pj = PredictionJobDataClass(
    **pj,
    default_modelspecs=ModelSpecificationDataClass(
        id=1,
    )
)

In [ ]:
from IPython.lib.display import IFrame
import random

forecasts = pd.DataFrame()
forecasts_list = []

for i, experiment in tqdm(enumerate(experiments)):
    print(f"Starting experiment: {experiment['name']}")
    #############
    # Data prep
    # Update pj
    pj["id"] = i + random.randint(0, 1000000)
    pj["description"] = experiment["name"]
    pj["default_modelspecs"]["id"] = pj["id"]

    # Split train and test
    train = experiment["dataset"].iloc[: -experiment["test_length"]]
    realised = experiment["dataset"].iloc[-experiment["test_length"]:]["load"]

    test = experiment["dataset"].copy(deep=True)
    # Set the test data to NaN for the last test_length rows
    test.iloc[-experiment["test_length"]:, 0] = (
        np.nan
    )  # This assumes the load column is the first one!
    # For forecasting we use the last 14 days of data as historical data, which is used for the lagged features
    # So we select the last 14 days of the training data + the day to forecast
    test = test.iloc[-15 * 24 * 4:]

    ##############
    # train model
    models = train_model_pipeline(
        pj,
        train,
        check_old_model_age=False,
        mlflow_tracking_uri="./mlflow_trained_models",
        artifact_folder="./mlflow_artifacts",
    )

    #################
    # Generate forecast
    forecast = create_forecast_pipeline(
        pj,
        test,
        mlflow_tracking_uri="./mlflow_trained_models",
    )
    # Add realised to forecast
    forecast["load"] = realised
    # Only keep region that was actually forecasted
    forecast = forecast.iloc[-experiment["test_length"]:]

    #######
    # Store forecast / Concatenate results
    forecasts = pd.concat([forecasts, forecast], axis=0)
    forecasts_list.append(forecast)
    
    display(IFrame('./mlflow_artifacts/{}/weight_plot.html'.format(pj['id']), width=800, height=400))

## Evaluate results

In [ ]:
# Get all column names that start with "quantile"
quantile_columns = [col for col in forecasts.columns if col.startswith("quantile")]

### Plot forecast against measurements

In [ ]:
##########
# Plot results
mean_forecasts = forecasts[["load", "forecast"] + quantile_columns].reset_index().groupby('index').mean()
fig = mean_forecasts[["load", "forecast"] + quantile_columns].plot()

fig.update_traces(line=dict(color="red", width=2), selector=lambda x: "load" in x.name)
fig.update_traces(
    line=dict(color="blue", width=2), selector=lambda x: "forecast" in x.name
)

# Show a green area between all quantile_columns
fig.update_traces(
    fill="tonexty",
    fillcolor="rgba(0,255,0,0.2)",
    selector=lambda x: "quantile" in x.name,
)

# Set all quantile traces to be green with half transparency
fig.update_traces(
    line=dict(color="green", width=1),
    opacity=0.5,
    selector=lambda x: "quantile" in x.name,
)

fig.show()

In [ ]:
def rmse_for_window(forecast_window, measurements, window_size):
    measurements_window = measurements.loc[forecast_window.index]
    mse = metrics.mean_squared_error(measurements_window, forecast_window)
    return np.sqrt(mse)


def mae_for_window(forecast_window, measurements, window_size):
    measurements_window = measurements.loc[forecast_window.index]
    return metrics.mean_absolute_error(measurements_window, forecast_window)

error_df = pd.DataFrame()

for fc in forecasts_list:
    forecast = fc[f"quantile_P{percentile}"]
    measurements = fc["load"]
    rmse = forecast.rolling(window=window_size).apply(
        rmse_for_window, args=(measurements, window_size)
    )
    mae = forecast.rolling(window=window_size).apply(
        mae_for_window, args=(measurements, window_size)
    )
    
    error_df_single = pd.concat([rmse, mae], axis=1)
    # Rename columns to "rmse" and "mae"
    error_df_single.columns = ["rmse", "mae"]
    error_df = pd.concat([error_df, error_df_single], axis=0)
    
error_df = error_df.reset_index().groupby('index').mean()

In [ ]:
forecasts

### rMAE

In [ ]:
mean_rmae = 0

# calculate rmae between forecast and measurements
for fc in forecasts_list:
    forecast = fc[f"quantile_P{percentile}"]
    measurements = fc["load"]
    
    rmae = np.sqrt(metrics.mean_squared_error(measurements, forecast))
    print(f"rMAE: {rmae}")
    mean_rmae += rmae

mean_rmae /= len(forecasts_list)
print(f'Mean rMAE: {mean_rmae}')

In [ ]:
if PLOT_ROLLING_WINDOW:
    error_df[["rmse", "mae"]].plot()

In [ ]:
def get_peaks_from_measurements(
        measurements,
        thresholds_pos: [int],
        thresholds_neg: [int],
        positive_peaks: bool = True,
        peak_distance: int = 12,
) -> [int]:
    """

    Args:
        measurements (_type_): _description_
        thresholds_pos (int]): _description_
        thresholds_neg (int]): _description_
        positive_peaks (bool, optional): _description_. Defaults to True.
        peak_distance (int, optional): Sample distance between peaks. Defaults to 12 (3 * 4 samples = 3 hours).

    Returns:
        _type_: _description_
    """

    # measurements = measurements.droplevel([1], axis=1)

    # Define constants

    peak_distance = 4 * 3  # samples are 15min, 3 hour distance

    # Detect peaks
    if positive_peaks:
        peaks = [
            (
                "terminal_id",
                signal.find_peaks(
                    measurements, height=thresholds_pos, distance=peak_distance
                ),
            )
        ]
    else:
        peaks = [
            (
                "terminal_id",
                signal.find_peaks(
                    -measurements, height=-thresholds_neg, distance=peak_distance
                ),
            )
        ]

    return peaks


def is_missed_peak_in_window(
        peak_measurement_window, peak_forecast_window, miss_threshold: int
):
    mape = metrics.mean_absolute_error(peak_measurement_window, peak_forecast_window)
    return mape > miss_threshold


def is_missed_peak(
        comp_df, peak_index: int, peak_evaluation_width: int, mape_threshold: int
):
    min_index = max(peak_index - peak_evaluation_width, 0)
    max_index = min(peak_index + peak_evaluation_width, len(comp_df))
    measurement_window = comp_df.loc[:, "Measured"][min_index:max_index]
    forecast_window = comp_df.loc[:, "Forecast"][measurement_window.index]
    return is_missed_peak_in_window(measurement_window, forecast_window, mape_threshold)


def get_missed_peaks(
        comp_df, peak_index_list: [int], peak_evaluation_width: int, mape_threshold: int
):
    return [
        peak
        for peak in peak_index_list
        if is_missed_peak(comp_df, peak, peak_evaluation_width, mape_threshold)
    ]


def plot_missed_peaks(
        comp_df,
        peak_index_list: [int],
        peak_evaluation_width: int,
        mape_threshold: int,
        subplots,
        plot_index,
):
    line_graph = px.line(comp_df)
    for trace in range(len(line_graph["data"])):
        subplots.append_trace(line_graph["data"][trace], row=plot_index, col=1)

    total_missed = 0
    for peak in peak_index_list:
        min_index = max(peak - peak_evaluation_width, 0)
        max_index = min(peak + peak_evaluation_width, len(comp_df))
        measurement_window = comp_df.loc[:, "Measured"][min_index:max_index]
        forecast_window = comp_df.loc[:, "Forecast"][measurement_window.index]

        start_rectangle = measurement_window.index[0]
        end_rectangle = measurement_window.index[-1]

        # Plot red area around missed peak
        if is_missed_peak_in_window(
                measurement_window, forecast_window, mape_threshold
        ):
            total_missed += 1
            subplots.add_vrect(
                x0=start_rectangle,
                x1=end_rectangle,
                line_width=0,
                fillcolor="red",
                opacity=0.2,
                row=plot_index,
                col=1,
            )
        else:
            subplots.add_vrect(
                x0=start_rectangle,
                x1=end_rectangle,
                line_width=0,
                fillcolor="blue",
                opacity=0.1,
                row=plot_index,
                col=1,
            )

    return total_missed

In [ ]:
forecast_percentile = percentile

In [ ]:
peaks_pos_list = []
peaks_neg_list = []

for fc in forecasts_list:
    measurements = fc["load"]


    thresholds_pos = measurements.quantile(load_threshold_max_quantile)
    thresholds_neg = measurements.quantile(load_threshold_min_quantile)
    
    peaks_pos = get_peaks_from_measurements(
        measurements,
        thresholds_pos=thresholds_pos,
        thresholds_neg=thresholds_neg,
        positive_peaks=True,
        peak_distance=peak_distance,
    )
    peaks_neg = get_peaks_from_measurements(
        measurements,
        thresholds_pos=thresholds_pos,
        thresholds_neg=thresholds_neg,
        positive_peaks=False,
        peak_distance=peak_distance,
    )
    
    peaks_pos_list.append(peaks_pos)
    peaks_neg_list.append(peaks_neg)

In [ ]:
if PLOT_POSITIVE_PEAKS:
    fig = subplots.make_subplots(rows=1, cols=1)
    
    avg_missed_pos = 0

    for i, fc in enumerate(forecasts_list):
        peaks_pos = peaks_pos_list[i]
        measurements = fc['load']
        print(i)
        
        for i, peak_list in enumerate(peaks_pos):
            terminal_id, peaks = peak_list
            measurements_for_terminal = measurements.rename("Measured")
            forecasts_for_terminal = fc[f"quantile_P{forecast_percentile}"].rename(
                "Forecast"
            )
    
            comp_df = pd.concat([measurements_for_terminal, forecasts_for_terminal], axis=1)
            print(comp_df.isnull().values.any())
            total_missed_pos = plot_missed_peaks(
                comp_df, peaks[0], peak_evaluation_width, peak_miss_threshold, fig, i + 1
            )

            print(f"Missed positive peaks: {total_missed_pos}")
            avg_missed_pos += total_missed_pos / len(forecasts_list)

        fig.show()
    
    print(f'Avg missed pos: {avg_missed_pos}')

In [ ]:
if PLOT_NEGATIVE_PEAKS:
    fig = subplots.make_subplots(rows=1, cols=1)

    for i, peak_list in enumerate(peaks_neg):
        terminal_id, peaks = peak_list
        measurements_for_terminal = measurements.rename("Measured")
        forecasts_for_terminal = forecasts[f"quantile_P{forecast_percentile}"].rename(
            "Forecast"
        )

        comp_df = pd.concat([measurements_for_terminal, forecasts_for_terminal], axis=1)
        print(comp_df.isnull().values.any())
        total_missed_pos = plot_missed_peaks(
            comp_df, peaks[0], peak_evaluation_width, peak_miss_threshold, fig, i + 1
        )

        print(f"Missed negative peaks: {total_missed_pos}")

    fig.show()

In [ ]:
if compare_positive_peaks:
    detected_peaks = peaks_pos_list[0]
else:
    detected_peaks = peaks_neg

In [ ]:
for i, fc in enumerate(forecasts_list):
    measurements = fc['load']
    
    forecasts_all_percentiles = fc[quantile_columns]
    terminals = ["terminal_id"]
    percentiles = quantile_columns

    missed_peaks_list = {
        terminal: {percentile: np.NaN for percentile in percentiles}
        for terminal in terminals
    }

    for percentile in percentiles:
        forecast_single_percentile = forecasts_all_percentiles[percentile]

        for i, peak_list in enumerate(detected_peaks):
            terminal_id, peaks = peak_list
            measurements_for_terminal = measurements.rename("Measured")
            forecasts_for_terminal = forecast_single_percentile.rename("Forecast")

            comp_df = pd.concat([measurements_for_terminal, forecasts_for_terminal], axis=1)
            missed_peaks_single_percentile_term = get_missed_peaks(
                comp_df, peaks[0], peak_evaluation_width, peak_miss_threshold
            )
            missed_peaks_list[terminal_id][percentile] = len(
                missed_peaks_single_percentile_term
            )

    missed_peaks_df = pd.DataFrame(missed_peaks_list)

    fig = subplots.make_subplots(
        rows=1,
        shared_yaxes="all",
        x_title="Percentile",
        y_title="Number of missed peaks",
        subplot_titles=terminals,
    )
    for i, terminal in enumerate(terminals):
        fig.add_trace(
            go.Bar(x=percentiles, y=missed_peaks_df[terminal], name=terminal),
            row=i + 1,
            col=1,
        )

    fig.update_layout(title_text="Missed peaks per terminal and percentile", height=1000)
    fig.show()